In [14]:
import pandas as pd
import numpy as np

PATH = "../data/interim/order_lines_canonical.parquet"
ol = pd.read_parquet(PATH)


pd.set_option("display.max_colwidth", None)

ol.shape, ol.columns.tolist()[:30]


((865553, 22),
 ['order_id',
  'anon',
  'date',
  'source',
  'amount',
  'raw_products',
  'clean_products',
  'item_name',
  'qty',
  'sku',
  'decode_method',
  'matched_key',
  'bottles',
  'pitchers',
  'filters_pitcher',
  'filters_bottle',
  'filters_unknown',
  'filters_bottle_included',
  'filters_pitcher_included',
  'uncertain',
  'total_bottle_filters',
  'total_pitcher_filters'])

In [16]:
sku_map = pd.read_parquet("../data/interim/sku_map.parquet")
sku_map.shape, sku_map.columns.tolist()


((1112, 8),
 ['produkty_clean',
  'Produkty',
  'RODZAJ',
  'ILOŚĆ FILTRÓW',
  'BAZOWY',
  'MATRIX NAZWA',
  'MATRIX GRUPA PRODUKTOWA',
  'CZY JEST W MATRIXIE'])

In [18]:
# choose matrix columns you want to carry forward
MATRIX_COLS = [
    "MATRIX NAZWA",
    "MATRIX GRUPA PRODUKTOWA",
    'ILOŚĆ FILTRÓW',
    

]

# keep only the join key + matrix columns, and deduplicate
sku_dim = (
    sku_map[["produkty_clean"] + MATRIX_COLS]
    .drop_duplicates("produkty_clean")
)

ol2 = ol.merge(
    sku_dim,
    how="left",
    left_on="matched_key",
    right_on="produkty_clean",
)

ol2[["matched_key", "produkty_clean"] + MATRIX_COLS].head(10)


,matched_key,produkty_clean,MATRIX NAZWA,MATRIX GRUPA PRODUKTOWA,ILOŚĆ FILTRÓW
0,"butelka filtrująca dafi solid 0,7 l szafirowa + filtr węglowy","butelka filtrująca dafi solid 0,7 l szafirowa + filtr węglowy","SOLID 0,7 1F",03_butelki filtrujące SOLID,1.0
1,"rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym","rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym","RURKA TRITANOWA DO SOLID 0,7",07_akcesoria do Soft/Solid,NaN
2,zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy,zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,3.0
3,"butelka filtrująca dafi soft 0,5 l limonkowa + filtr węglowy","butelka filtrująca dafi soft 0,5 l limonkowa + filtr węglowy","SOFT 0,5 1F",02_butelki filtrujące SOFT,1.0
4,"butelka filtrująca dafi soft 0,5 l niebiańska + filtr węglowy","butelka filtrująca dafi soft 0,5 l niebiańska + filtr węglowy","SOFT 0,5 1F",02_butelki filtrujące SOFT,1.0
5,"dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 kw z baterią białą","dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 kw z baterią białą",PRZEPŁYWOWY PODGRZEWACZ - NADUMYLAWKOWY,26_podgrzewacze przepływowe,NaN
6,zakrętka do butelki filtrującej dafi solid uchwyt flamingowy,zakrętka do butelki filtrującej dafi solid uchwyt flamingowy,ZAKRĘTKA DO SOLID UCHWYT / PRZYCISK / NULL,07_akcesoria do Soft/Solid,NaN
7,"rurka na filtr do butelki filtrującej dafi solid 0,5 l flamingowa","rurka na filtr do butelki filtrującej dafi solid 0,5 l flamingowa","RURKA TRITANOWA DO SOLID 0,5",07_akcesoria do Soft/Solid,NaN
8,"rurka na filtr do butelki filtrującej dafi solid 0,5 l turkusowa","rurka na filtr do butelki filtrującej dafi solid 0,5 l turkusowa","RURKA TRITANOWA DO SOLID 0,5",07_akcesoria do Soft/Solid,NaN
9,zestaw 3 filtry do butelki filtrującej dafi soft i solid turkusowy,zestaw 3 filtry do butelki filtrującej dafi soft i solid turkusowy,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,3.0


In [43]:
len(ol2)

865553

In [ ]:
df = ol2.copy()

# fill missing matrix labels with a clear placeholder
df["MATRIX NAZWA"] = df["MATRIX NAZWA"].fillna("__INFERRED__")
df["MATRIX GRUPA PRODUKTOWA"] = df["MATRIX GRUPA PRODUKTOWA"].fillna("__INFERRED__")

mask_inferred = df["MATRIX NAZWA"].eq("__INFERRED__")
mask_inferred.mean(), mask_inferred.sum()



(0.006683588411108274, 5785)

In [33]:
BOTTLE_FILTER_GROUP = "06_filtry do butelek Soft i Solid"
PITCHER_FILTER_GROUP = "__PITCHER_FILTERS__"


In [35]:
mask_pure_bottle_filters = (
    mask_inferred
    & (df["filters_bottle"] > 0)
    & (df["filters_bottle_included"] == 0)
    & (df["filters_pitcher"] == 0)
    & (df["filters_pitcher_included"] == 0)
)

mask_pure_bottle_filters.sum()


18

In [36]:
df.loc[mask_pure_bottle_filters, "MATRIX GRUPA PRODUKTOWA"] = (
    "06_filtry do butelek Soft i Solid"
)

df.loc[mask_pure_bottle_filters, "MATRIX NAZWA"] = "1 FILTR BUTELKOWY"


In [37]:
df.loc[mask_pure_bottle_filters, "matrix_source"] = "rules_backfilled_pure_filters"


In [38]:
df["matrix_source"] = df["matrix_source"].fillna("unmapped_inferred")


In [39]:
df.loc[
    mask_pure_bottle_filters,
    [
        "item_name",
        "filters_bottle",
        "filters_bottle_included",
        "filters_pitcher",
        "MATRIX NAZWA",
        "MATRIX GRUPA PRODUKTOWA",
        "matrix_source",
    ],
].head(10)


,item_name,filters_bottle,filters_bottle_included,filters_pitcher,MATRIX NAZWA,MATRIX GRUPA PRODUKTOWA,matrix_source
16127,3 filtry do butelki dafi soft i solid białe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
16718,3 filtry do butelki dafi soft i solid flamingowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
17340,3 filtry do butelki dafi soft i solid bursztynowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
17613,3 filtry do butelki dafi soft i solid jagodowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
17706,3 filtry do butelki dafi soft i solid szafirowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
18173,3 filtry do butelki dafi soft i solid białe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
18495,3 filtry do butelki dafi soft i solid cytrynowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
18684,3 filtry do butelki dafi soft i solid turkusowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
18685,3 filtry do butelki dafi soft i solid waniliowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters
19019,3 filtry do butelki dafi soft i solid szafirowe,3,0,0,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,rules_backfilled_pure_filters


In [45]:
retention_df = df[
    df["MATRIX GRUPA PRODUKTOWA"].notna()
    & (df["MATRIX GRUPA PRODUKTOWA"] != "__INFERRED__")
]


In [67]:
df["units_purchased"] = df["qty"].astype(float)


In [68]:
mask_sku = df["matrix_source"].eq("sku_lookup")

df.loc[mask_sku, "matrix_qty"] = (
    df.loc[mask_sku, "qty"]
    * df.loc[mask_sku, "ILOŚĆ FILTRÓW"].fillna(1)
)


In [69]:
mask_sku = df["matrix_source"].eq("sku_lookup")

df.loc[mask_sku, "matrix_qty"] = (
    df.loc[mask_sku, "qty"]
    * df.loc[mask_sku, "ILOŚĆ FILTRÓW"].fillna(1)
)


In [70]:
mask_rules = df["matrix_source"].eq("rules_backfilled_pure_filters")

df.loc[mask_rules, "matrix_qty"] = df.loc[mask_rules, "total_bottle_filters"]


In [71]:
df["matrix_source"] = np.where(
    df["decode_method"].eq("sku_lookup"),
    "sku_lookup",
    "unmapped_inferred"
)

# keep your special backfilled label if you already set it earlier
# (only overwrite where it isn't already backfilled)
df.loc[df["matrix_source"].ne("sku_lookup") & df["matrix_source"].ne("rules_backfilled_pure_filters"), "matrix_source"] = "unmapped_inferred"

df["matrix_source"].value_counts(dropna=False)


matrix_source
sku_lookup           859768
unmapped_inferred      5785
Name: count, dtype: int64

In [72]:
cust_day_group["matrix_qty"].value_counts(dropna=False).head(10)


matrix_qty
0.0    677485
Name: count, dtype: int64

## Building `cust_day_group`: customer × day × product-group timeline

At this stage, we construct a **retention-ready event table** that captures *when* a customer acquires a given **product group** and *in what quantity*.

This table is the foundation for:
- inter-purchase interval analysis
- replacement-cycle estimation
- churn / retention modeling
- downstream LTV features

### Key design decisions

**1. Retention-relevant rows only**

Not every decoded order line should participate in retention analysis.  
We explicitly restrict to rows that represent **true product acquisition**:

- `matrix_source == "sku_lookup"`  
  → SKU-matched products with trusted matrix metadata
- `matrix_source == "rules_backfilled_pure_filters"`  
  → non-SKU rows representing *pure* bottle-filter purchases (deterministically inferred)

Accessories, ambiguous items, and non-SKU devices are excluded from retention modeling but remain in the canonical dataset for auditability.

This logic is captured by the boolean flag:
```python
is_retention_relevant


In [81]:
cust_day_group = (
    df[df["is_retention_relevant"]]
    .groupby(
        ["anon", "date", "MATRIX GRUPA PRODUKTOWA"],
        as_index=False
    )
    .agg({
        "matrix_qty": "sum",
        "ILOŚĆ FILTRÓW": "sum",
        "MATRIX NAZWA": "first",
    })
    .sort_values(["anon", "date"])
)



cust_day_group.head(10), cust_day_group.shape


(           anon     date            MATRIX GRUPA PRODUKTOWA  matrix_qty  \
 0  ANON_0000001  11/9/22        03_butelki filtrujące SOLID         1.0   
 1  ANON_0000001  11/9/22  06_filtry do butelek Soft i Solid         3.0   
 2  ANON_0000001  11/9/22         07_akcesoria do Soft/Solid         1.0   
 3  ANON_0000002  11/9/22         02_butelki filtrujące SOFT         2.0   
 4  ANON_0000003  11/9/22        26_podgrzewacze przepływowe         1.0   
 5  ANON_0000004  11/9/22         07_akcesoria do Soft/Solid         3.0   
 6  ANON_0000005  11/9/22  06_filtry do butelek Soft i Solid         3.0   
 7  ANON_0000006  11/9/22        03_butelki filtrujące SOLID         1.0   
 8  ANON_0000007  11/9/22         02_butelki filtrujące SOFT         2.0   
 9  ANON_0000007  11/9/22  06_filtry do butelek Soft i Solid         2.0   
 
    ILOŚĆ FILTRÓW                                MATRIX NAZWA  
 0            1.0                                SOLID 0,7 1F  
 1            3.0                 

In [82]:
# how many real purchase rows do we have?
cust_day_group["is_purchase"] = cust_day_group["matrix_qty"] > 0

cust_day_group["is_purchase"].value_counts(normalize=True)


is_purchase
True    1.0
Name: proportion, dtype: float64

In [83]:
# Notebook 04 — FINAL CELL

from pathlib import Path

OUT = Path("../data/interim")
OUT.mkdir(parents=True, exist_ok=True)

path = OUT / "cust_day_group.parquet"
cust_day_group.to_parquet(path, index=False)

print(f"Saved cust_day_group → {path}")
print("Rows:", len(cust_day_group))
print("Columns:", cust_day_group.columns.tolist())


Saved cust_day_group → ../data/interim/cust_day_group.parquet
Rows: 677485
Columns: ['anon', 'date', 'MATRIX GRUPA PRODUKTOWA', 'matrix_qty', 'ILOŚĆ FILTRÓW', 'MATRIX NAZWA', 'is_purchase']


## Output: `cust_day_group`

This notebook produces `cust_day_group.parquet`, the canonical daily purchase table
used as input for retention and LTV modeling.

Grain:
- one row per (anon, date, MATRIX GRUPA PRODUKTOWA)

Key fields:
- `matrix_qty` — economically meaningful quantity
  - SKU rows: `qty × ILOŚĆ FILTRÓW`
  - Non-SKU filter-only rows: backfilled conservatively
- `matrix_source` — SKU / backfill provenance
- `is_retention_relevant` — whether the row should be used for replacement modeling

All decoding, SKU logic, and safety assumptions stop here.
Downstream notebooks must not reinterpret product semantics.
